# Tugas Besar IF3270 Pembelajaran Mesin Artificial Neural Network
Bagian A - Implementasi Forward Propagation untuk Feed Forward Neural Network

## Kontributor:

- Kartini Copa (13521026)

- Muhammad Equilibrie Fajria (13521047)

- M Farrel Danendra Rachim (13521048)

- Fakih Anugerah Pratama (13521091)

## Activation Function

In [ ]:
import math
import numpy as np

In [ ]:
def fLinear(net):
    return net

def fRelu(net):
    return max(0, net)

def fSigmoid(net):
    return 1 / (1 + np.exp(-net))

def fSoftmax(net):
    return np.exp(net)

activationFunctions = {
    'linear': fLinear,
    'relu': fRelu,
    'sigmoid': fSigmoid,
    'softmax': fSoftmax
}

## Loader

In [ ]:
import json

class Loader:
    def load_data(filename):
        base_path = '/work/'
        with open(base_path + filename, 'r') as file:
            data = json.load(file)
    
        # model data
        __model = data['case']['model']

        ## input layer
        input_size = __model['input_size']

        ## hidden and output layer
        __layers = __model['layers']
        layers = [[_layer['number_of_neurons'], _layer['activation_function']] for _layer in __layers]

        ## weights
        weights = data['case']['weights']

        # test
        ## input data
        input_data = data['case']['input']

        ## output expectation
        expected_output = data['expect']['output']
        max_sse = data['expect']['max_sse']


        # return loaded data
        return input_size, layers, weights, input_data, expected_output, max_sse

input_size, layers, weights, input_data, expected_output, max_sse = Loader.load_data('multilayer.json')

print("Input Size:")
print(input_size)
print("Layers")
print(layers)
print("\nWeights:")
print(weights)
print("Input Data:")
print(input_data)
print("\nExpected Output:")
print(expected_output)
print("\nsse output:")
print(max_sse)

Input Size:
3
Layers
[[4, 'relu'], [3, 'relu'], [2, 'relu'], [1, 'sigmoid']]

Weights:
[[[0.1, 0.2, 0.3, -1.2], [-0.5, 0.6, 0.7, 0.5], [0.9, 1.0, -1.1, -1.0], [1.3, 1.4, 1.5, 0.1]], [[0.1, 0.1, 0.3], [-0.4, 0.5, 0.6], [0.7, 0.4, -0.9], [0.2, 0.3, 0.4], [-0.1, 0.2, 0.1]], [[0.1, 0.2], [-0.3, 0.4], [0.6, 0.1], [0.1, -0.4]], [[0.1], [-0.2], [0.3]]]
Input Data:
[[-1.0, 0.5, 0.8]]

Expected Output:
[[0.4846748]]

sse output:
1e-06


## Node

In [ ]:
class Node:
    def __init__(self, nodeId : int, layerId : int, weight : [float] = None, isBias: bool = False) -> None:
        self.nodeId = nodeId
        self.layerId = layerId
        self.weight = weight
        self.bias = isBias

    def showInfo(self) -> None:
        if (self.bias):
            print(f"      Bias:")
        else:
            print(f"      Node {self.nodeId}:")
        print(f"         Weight: {self.weight}")

## Layer

In [ ]:
# layerType = "hidden", "input", "output"
HIDDEN = "hidden"
INPUT = "input"
OUTPUT = "output"

class Layer:
    def __init__(self, layerId : int, neuronCount: int, nodeList, layerType: str, activationFunction : str = None) -> None:
        self.layerId = layerId
        self.neuronCount = neuronCount
        self.nodeList = nodeList
        self.layerType = layerType
        self.activationFunction = activationFunction

    def calculateNet(self, inputList):
        net = []
        nextNeuronSize = len(self.nodeList[0].weight)

        for i in range (nextNeuronSize):
            sum = self.nodeList[0].weight[i]
            for j in range (len(inputList)):
                sum += self.nodeList[j+1].weight[i] * inputList[j]
            
            net.append(sum)

        return net

    def showInfo(self) -> None:
        print(f"Layer {self.layerId}:")
        print(f"   Type: {self.layerType}")
        print(f"   Neuron Count: {self.neuronCount}")
        print(f"   Activation Function: {self.activationFunction}")
        print(f"   Nodes:")
        for node in self.nodeList:
            if (self.layerType != OUTPUT or (not node.bias)):
                node.showInfo()
        print()

## Model

In [ ]:
class Model:
    def __init__(self, input_size, layers, weights):
        self.layers = []

        # input layer
        _nodes = [Node(i, 0, weights[0][i], True if i == 0 else False) for i in range(input_size + 1)]
        self.layers.append(Layer(0, input_size, _nodes, INPUT))

        # hidden and output layer
        for j in range (len(layers)):
            layerId = j + 1
            neuronCount = layers[j][0]
            activationFunction = layers[j][1]
            layerType = ""

            if (j != len(layers)-1):
                _nodes = [Node(i, layerId, weights[j+1][i], True if i == 0 else False) for i in range(neuronCount + 1)]
                layerType = HIDDEN
            else:
                _nodes = [Node(i, j+1, [], True if i == 0 else False) for i in range (neuronCount + 1)]
                layerType = OUTPUT
                
            self.layers.append(Layer(layerId, neuronCount, _nodes, layerType, activationFunction))

## FFNN

In [ ]:
import copy

class FFNN:
    def __init__(self, model) -> None:
        self.layerList = model.layers

    # Predict a single input
    def predict(self, inputList):  
        output = copy.deepcopy(inputList)
        net = [] 

        for layer in self.layerList:
            if (layer.layerType != INPUT):
                output = [activationFunctions[layer.activationFunction](x) for x in net]
                if (layer.activationFunction == "softmax"):
                    sigma = sum(output)
                    for i in range (len(output)):
                       output[i] /= sigma
            if (layer.layerType != OUTPUT):
                net = layer.calculateNet(output)

        return output

    # Predict multiple input
    def predictMultiple(self, inputList):
        res = []
        for val in inputList:
            res.append(self.predict(val))
        return res

    def showInfo(self) -> None:
        print("--------------------------------------------------------------------------------")
        print("ANN Model:")
        print()
        for layer in self.layerList:
            layer.showInfo()
        print("--------------------------------------------------------------------------------")

## Main Program

In [ ]:
input_size, layers, weights, input_data, expected_output, max_sse = Loader.load_data('tes.json')
model = Model(input_size, layers, weights)

ffnn = FFNN(model)
ffnn.showInfo()

fm = ffnn.predictMultiple(input_data)
print("Predictions:")
print(fm)
print()
print("Expected:")
print(expected_output)
print()

--------------------------------------------------------------------------------
ANN Model:

Layer 0:
   Type: input
   Neuron Count: 2
   Activation Function: None
   Nodes:
      Bias:
         Weight: [30]
      Node 1:
         Weight: [-20]
      Node 2:
         Weight: [-20]

Layer 1:
   Type: hidden
   Neuron Count: 1
   Activation Function: sigmoid
   Nodes:
      Bias:
         Weight: [-25, 20, -10]
      Node 1:
         Weight: [20, -10, 10]

Layer 2:
   Type: output
   Neuron Count: 3
   Activation Function: sigmoid
   Nodes:
      Node 1:
         Weight: []
      Node 2:
         Weight: []
      Node 3:
         Weight: []

--------------------------------------------------------------------------------
Predictions:
[[0.006686817475015197, 0.9999546227353499, 0.49988650533019335]]

Expected:
[[0, 0, 0]]



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=afc3e09d-2ff9-42fe-8178-5c8617fbb136' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>